In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from tensorflow.contrib import learn

### Parameters

In [2]:
# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 100, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 100, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 50, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_EVERY=50
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=50
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=100
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos



### Load data

In [3]:
# Load data
print("Loading data...")
x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary -> learn library preprocessing.VocabularyProcessor를 이용해서 
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066


### CNN Model

In [14]:
class TextCNN(object):
    def __init__(self,
                sequence_length,
                num_classes,
                vocab_size,
                embedding_size,
                filter_sizes,
                num_filters):
        
        
        # 1) placeholder(x = [None, sequence_length])
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length])
        self.input_y = tf.placeholder(tf.float32, [None, num_classes])
        self.dropout_keep_prob = tf.placeholder(tf.float32)
        
        l2_loss = tf.constant(0.0)
        
        
        # 2) embedding layer(tensor = [None, sequence_length, embedding_size])
        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            

        # 3) convolution and max-pooling layers
        pooled_outputs = []
        for i,filter_size in enumerate(filter_sizes):
            with tf.name_scope('conv-maxpool-%s'%filter_size):
                # convolution layer(tensor = [None, sequence_length-filter_size+1, 1, num_filter])
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]))
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1,1,1,1],
                    padding='VALID'
                )
                
                # apply nonlinearlity
                h = tf.nn.relu(tf.nn.bias_add(conv,b))
                
                # max-pooling over the outputs(tensor = [None, 1, 1, num_filter])
                pooled = tf.nn.max_pool(
                    h,
                    ksize = [1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1,1,1,1],
                    padding='VALID'
                )
                pooled_outputs.append(pooled)

                
        # 4) combine all the pooled features(tensor = [None, num_filter*len(filter_sizes)])
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs,3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        
        # 5) dropout layer(tensor = [None, num_filters_total])
        with tf.name_scope('dropout'):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

            
        # 6) output layer(fully connected layer, tensor = [None, num_classes])
        with tf.name_scope('output'):
            W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1))
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b)
            self.predictions = tf.argmax(self.scores, 1)

            
        # 7) loss
        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits = self.scores, labels = self.input_y)
            self.loss = tf.reduce_mean(losses)

            
        # 8) accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y,1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, 'float'))

### Training Procedure

In [15]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
    allow_soft_placement = FLAGS.allow_soft_placement,
    log_device_placement = FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    
    with sess.as_default():
        cnn = TextCNN(sequence_length=x_train.shape[1],
                     num_classes=y_train.shape[1],
                     vocab_size=len(vocab_processor.vocabulary_),
                     embedding_size=FLAGS.embedding_dim,
                     filter_sizes=list(map(int, FLAGS.filter_sizes.split(','))),
                     num_filters=FLAGS.num_filters
                     )
        
        # define Training procedure
        global_step = tf.Variable(0,trainable=False)
        optimizer = tf.train.AdamOptimizer(0.001)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # keep track of gradient values and sparsity
        grad_summaries = []
        for g,v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram('{}/grad/hist'.format(v.name), g)
                sparsity_summary = tf.summary.scalar('{}/grad/sparsity'.format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)
        
        # output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, 'runs', timestamp))
        print('writing to {}\n'.format(out_dir))
        
        # summaries for loss and accuracy
        loss_summary = tf.summary.scalar('loss', cnn.loss)
        acc_summary = tf.summary.scalar('accuracy', cnn.accuracy)
        
        # train summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, 'summaries', 'train')
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, 'summaries', 'dev')
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # checkpoint directory
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, 'checkpoints'))
        checkpoint_prefix = os.path.join(checkpoint_dir, 'model')
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep = FLAGS.num_checkpoints)
        
        # write voca
        vocab_processor.save(os.path.join(out_dir, 'vocab'))
        
        # initialize
        sess.run(tf.global_variables_initializer())
        
        # train
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                
        # generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs
        )
        # training loop
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


INFO:tensorflow:Summary name embedding/Variable:0/grad/hist is illegal; using embedding/Variable_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/Variable:0/grad/sparsity is illegal; using embedding/Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/Variable:0/grad/hist is illegal; using conv-maxpool-3/Variable_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/Variable:0/grad/sparsity is illegal; using conv-maxpool-3/Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/Variable_1:0/grad/hist is illegal; using conv-maxpool-3/Variable_1_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/Variable_1:0/grad/sparsity is illegal; using conv-maxpool-3/Variable_1_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/Variable:0/grad/hist is illegal; using conv-maxpool-4/Variable_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/Variable:0/grad/sparsity is illegal; using conv-

### tensorboard

tensorboard --logdir /PATH_TO_CODE/runs/1449760558/summaries/
